# Fill Missing Values Using Random Forest

In [1]:
## Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Import Dataset
Data = pd.read_csv('C:/Users/medipalle.tendulkar/Desktop/Kaggle DataSets/train.csv',index_col='PassengerId')
Data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Preprocessing

In [3]:
# Missing Values

Data.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [4]:
Data = Data.drop(columns = ['Cabin'],axis=1)

In [5]:
Data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [6]:
# Lets check Embarked Feature
Data.Embarked.value_counts()


S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [7]:
# As 'S' is the Most frequent category iam going to replace the null values with Most frequent Value i.e, Median

Data['Embarked'].fillna(Data['Embarked'].value_counts().index[0], inplace = True)

In [8]:
DataWithAge = Data[pd.isnull(Data['Age']) == False]
DataWithoutAge = Data[pd.isnull(Data['Age'])]

In [9]:
DataWithAge.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [10]:
DataWithoutAge.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q
18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,S
20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,C
27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,C
29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,Q


In [11]:
print(DataWithAge.shape,DataWithoutAge.shape)

(714, 10) (177, 10)


In [12]:
# As we Focused on Filling Missing values iam selecting only features that are important.
Features = ['Survived','Pclass','Age','SibSp','Parch','Fare']

In [13]:
# Additionally, categorical variables must be encoded as numeric values. This task can be done using one-hot encoding

one_hot_embarked = pd.get_dummies(DataWithAge['Embarked'],drop_first=True)
one_hot_sex = pd.get_dummies(DataWithAge['Sex'],drop_first=True)
DataWithAge = DataWithAge[Features]
TrainSet = pd.concat([DataWithAge, one_hot_sex, one_hot_embarked], axis = 1)

one_hot_embarked = pd.get_dummies(DataWithoutAge['Embarked'],drop_first=True)
one_hot_sex = pd.get_dummies(DataWithoutAge['Sex'],drop_first=True)
DataWithoutAge = DataWithoutAge[Features]
TestSet = pd.concat([DataWithoutAge, one_hot_sex, one_hot_embarked], axis = 1)

In [14]:
TrainSet.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
PassengerId,,,,,,,,,
1,0,3,22.0,1,0,7.2500,1,0,1
2,1,1,38.0,1,0,71.2833,0,0,0
3,1,3,26.0,0,0,7.9250,0,0,1
4,1,1,35.0,1,0,53.1000,0,0,1
5,0,3,35.0,0,0,8.0500,1,0,1


In [15]:
TestSet.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
PassengerId,,,,,,,,,
6,0,3,NaN,0,0,8.4583,1,1,0
18,1,2,NaN,0,0,13.0000,1,0,1
20,1,3,NaN,0,0,7.2250,0,0,0
27,0,3,NaN,0,0,7.2250,1,0,0
29,1,3,NaN,0,0,7.8792,0,1,0


In [16]:
TrainSet.columns

Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q',
       'S'],
      dtype='object')

In [17]:
Independent_Features = ['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare', 'male', 'Q','S']

# Model Buliding

In [18]:
# Now the crucial part. It is the time to train the Random Forest regressor and predict the values of the “Age” column
from sklearn.ensemble import RandomForestRegressor
rf_age = RandomForestRegressor()
# Training
rf_age.fit(TrainSet[Independent_Features], TrainSet['Age'])

D:\DATA_SCIENCE\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [19]:
# Prediction

Predicted_Ages = rf_age.predict(TestSet[Independent_Features])

In [20]:
Predicted_Ages

array([33.9125    , 35.83      , 14.7       , 34.64916667, 18.7       ,
       27.34235313, 35.58      , 18.6       , 20.2       , 33.34385965,
       32.09474991, 37.3025    , 18.6       , 22.375     , 36.4       ,
       39.55      , 14.126     , 27.34235313, 32.09474991, 18.3       ,
       32.09474991, 32.09474991, 27.34235313, 26.97181818, 16.6       ,
       32.09474991, 44.465     ,  7.81666667, 19.8       , 31.07064325,
       24.81690476,  8.86333333, 22.        , 57.2       ,  6.4       ,
       15.15      , 29.7       , 48.48      , 15.4       , 44.465     ,
       18.6       ,  8.86333333, 36.0275    , 27.34235313,  5.06666667,
       38.675     , 20.65      , 15.4       , 31.07064325, 32.66666667,
       44.465     , 27.36666667, 51.2       , 18.6       , 35.27381313,
       57.2       , 39.55      , 37.83333333, 18.6       , 24.975     ,
       27.025     , 32.09474991, 30.        ,  8.86333333, 19.7       ,
       31.28      , 27.34235313, 27.7       , 58.8       , 34.64

In [21]:
TestSet['Age'] = Predicted_Ages.astype(int)
Titanic_set = TrainSet.append(TestSet)

In [22]:
# Final Dataset with No Null Values in Age.
Titanic_set.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
PassengerId,,,,,,,,,
1,0,3,22.0,1,0,7.2500,1,0,1
2,1,1,38.0,1,0,71.2833,0,0,0
3,1,3,26.0,0,0,7.9250,0,0,1
4,1,1,35.0,1,0,53.1000,0,0,1
5,0,3,35.0,0,0,8.0500,1,0,1


In [23]:
Titanic_set.shape

(891, 9)

In [24]:
Titanic_set.isnull().sum()

Survived    0
Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
male        0
Q           0
S           0
dtype: int64